In [34]:
import re
import colorama
import requests 
from bs4 import BeautifulSoup
import folium
import numpy as np
import pandas as pd


Limpieza de datos

In [35]:
df_conectividad_internet = pd.read_csv('Dataset/Conectividadalnternet_mapa.csv', encoding='utf-8')
df_conectividad_internet.head()

,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,DIALUP,FIBRAOPTICA,SATELITAL,WIRELESS,TELEFONIAFIJA,3G,4G,link,Latitud,Longitud
0,BUENOS AIRES,Bahía Blanca,Villa Bordeau,0,--,--,--,--,--,--,--,SI,SI,6056020.0,"-38,6472605094596","-62,3200223419024"
1,BUENOS AIRES,Bahía Blanca,Villa Espora,0,--,--,--,--,--,--,--,--,--,6056020.0,"-38,7765069529222","-62,1851833537179"
2,BUENOS AIRES,Balcarce,Balcarce,38376,SI,SI,SI,SI,--,SI,SI,SI,SI,6063010.0,"-37,8482779294345","-58,2551665841248"
3,BUENOS AIRES,Balcarce,Los Pinos,337,--,--,--,--,--,SI,SI,--,--,6063020.0,"-37,9412057603","-58,3225920150442"
4,BUENOS AIRES,Balcarce,Napaleofú,374,--,--,--,SI,--,SI,SI,SI,SI,6063030.0,"-37,6254980210026","-58,7461862359423"


In [36]:
#Eliminamos columna link
df_conectividad_internet.drop("link", axis=1, inplace=True)

In [37]:
df_conectividad_internet.isnull().sum()

Provincia        0
Partido          0
Localidad        0
Poblacion        0
ADSL             0
CABLEMODEM       0
DIALUP           0
FIBRAOPTICA      0
SATELITAL        0
WIRELESS         0
TELEFONIAFIJA    0
3G               0
4G               0
Latitud          0
Longitud         0
dtype: int64

In [38]:
df_conectividad_internet.dropna()

,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,DIALUP,FIBRAOPTICA,SATELITAL,WIRELESS,TELEFONIAFIJA,3G,4G,Latitud,Longitud
0,BUENOS AIRES,Bahía Blanca,Villa Bordeau,0,--,--,--,--,--,--,--,SI,SI,"-38,6472605094596","-62,3200223419024"
1,BUENOS AIRES,Bahía Blanca,Villa Espora,0,--,--,--,--,--,--,--,--,--,"-38,7765069529222","-62,1851833537179"
2,BUENOS AIRES,Balcarce,Balcarce,38376,SI,SI,SI,SI,--,SI,SI,SI,SI,"-37,8482779294345","-58,2551665841248"
3,BUENOS AIRES,Balcarce,Los Pinos,337,--,--,--,--,--,SI,SI,--,--,"-37,9412057603","-58,3225920150442"
4,BUENOS AIRES,Balcarce,Napaleofú,374,--,--,--,SI,--,SI,SI,SI,SI,"-37,6254980210026","-58,7461862359423"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,CHACO,Quitilipi,Quitilipi,24517,SI,--,--,SI,--,SI,SI,SI,SI,"-26,8732071732978","-60,2185241226399"
4308,CHACO,Quitilipi,Villa El Palmar,879,--,--,--,--,--,SI,--,--,SI,"-26,4551186226266","-60,1646165913523"
4309,CHACO,San Fernando,Barranqueras,54698,SI,SI,--,SI,--,SI,SI,SI,SI,"-27,4877739289761","-58,9327416886365"
4310,CHACO,San Fernando,Basail,1929,SI,SI,--,--,SI,SI,SI,SI,SI,"-27,8868655966917","-59,2791003619491"


In [39]:
df_conectividad_internet.replace("--", np.nan, inplace=True)

In [40]:
#Cambiamos coma por punto
df_conectividad_internet["Longitud"] = df_conectividad_internet["Longitud"].str.replace(",", ".")
df_conectividad_internet["Latitud"] = df_conectividad_internet["Latitud"].str.replace(",", ".")

In [41]:
# convertimos las columnas latitud y longitud en numericos
df_conectividad_internet["Latitud"] = pd.to_numeric(df_conectividad_internet["Latitud"], errors="coerce") 

df_conectividad_internet["Longitud"] = pd.to_numeric(df_conectividad_internet["Longitud"], errors="coerce")

In [42]:
df_conectividad_internet

,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,DIALUP,FIBRAOPTICA,SATELITAL,WIRELESS,TELEFONIAFIJA,3G,4G,Latitud,Longitud
0,BUENOS AIRES,Bahía Blanca,Villa Bordeau,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SI,SI,-38.647261,-62.320022
1,BUENOS AIRES,Bahía Blanca,Villa Espora,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-38.776507,-62.185183
2,BUENOS AIRES,Balcarce,Balcarce,38376,SI,SI,SI,SI,NaN,SI,SI,SI,SI,-37.848278,-58.255167
3,BUENOS AIRES,Balcarce,Los Pinos,337,NaN,NaN,NaN,NaN,NaN,SI,SI,NaN,NaN,-37.941206,-58.322592
4,BUENOS AIRES,Balcarce,Napaleofú,374,NaN,NaN,NaN,SI,NaN,SI,SI,SI,SI,-37.625498,-58.746186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,CHACO,Quitilipi,Quitilipi,24517,SI,NaN,NaN,SI,NaN,SI,SI,SI,SI,-26.873207,-60.218524
4308,CHACO,Quitilipi,Villa El Palmar,879,NaN,NaN,NaN,NaN,NaN,SI,NaN,NaN,SI,-26.455119,-60.164617
4309,CHACO,San Fernando,Barranqueras,54698,SI,SI,NaN,SI,NaN,SI,SI,SI,SI,-27.487774,-58.932742
4310,CHACO,San Fernando,Basail,1929,SI,SI,NaN,NaN,SI,SI,SI,SI,SI,-27.886866,-59.279100


In [50]:
#df_filtered = df_conectividad_internet.dropna(subset=['Latitud', 'Longitud'])


In [57]:
# Verificar datos faltantes en la columna 'Provincia'
provincias_grouped = df_conectividad_internet.groupby('Provincia')
provincias_con_discrepancia = []
provincias_nulas = df_conectividad_internet[df_conectividad_internet['Provincia'].isnull()]

In [58]:
for provincia, grupo in provincias_grouped:
    valores_unicos = grupo['Provincia'].unique()
    if len(valores_unicos) > 1:
        provincias_con_discrepancia.append((provincia, valores_unicos))

# Imprimir las provincias con discrepancias
print("Provincias con valores únicos encontrados:")
for provincia, valores_unicos in provincias_con_discrepancia:
    print(f"Provincia: {provincia}")
    print(f"Valores únicos encontrados: {valores_unicos}\n")

Provincias con valores únicos encontrados:


In [59]:
# Imprimir filas con datos faltantes en la columna 'Provincia'
print("Filas con datos faltantes en la columna 'Provincia':")
print(provincias_nulas)

Filas con datos faltantes en la columna 'Provincia':
Empty DataFrame
Columns: [Provincia, Partido, Localidad, Poblacion, ADSL, CABLEMODEM, DIALUP, FIBRAOPTICA, SATELITAL, WIRELESS, TELEFONIAFIJA, 3G, 4G, Latitud, Longitud]
Index: []


In [60]:
# Filtrar las conexiones disponibles
df_conexiones = df_conectividad_internet[['Localidad', 'Latitud', 'Longitud', 'ADSL', 'CABLEMODEM', 'DIALUP', 'FIBRAOPTICA', 'SATELITAL']].copy()
df_conexiones = df_conexiones[(df_conexiones['ADSL'] == 'SI') | (df_conexiones['CABLEMODEM'] == 'SI') | (df_conexiones['DIALUP'] == 'SI') | (df_conexiones['FIBRAOPTICA'] == 'SI') | (df_conexiones['SATELITAL'] == 'SI')]

CREAR MAPA

In [66]:
# Filtrar las conexiones disponibles
df_conexiones = df_conectividad_internet[['Localidad', 'Latitud', 'Longitud', 'Provincia', 'ADSL', 'CABLEMODEM', 'DIALUP', 'FIBRAOPTICA', 'SATELITAL']].copy()

df_conexiones = df_conexiones[(df_conexiones['ADSL'] == 'SI') | (df_conexiones['CABLEMODEM'] == 'SI') | (df_conexiones['DIALUP'] == 'SI') | (df_conexiones['FIBRAOPTICA'] == 'SI') | (df_conexiones['SATELITAL'] == 'SI')]

In [68]:
#Creamos un mapa vacío con una ubicación inicial
mapa = folium.Map(location=[-34.6037, -58.3816], zoom_start=6)

In [69]:
colores = {'ADSL': 'blue', 'CABLEMODEM': 'red', 'DIALUP': 'green', 'FIBRAOPTICA': 'orange', 'SATELITAL': 'purple'}


In [70]:
#Contamos los valores nulos en columna Latitud y Longitud
nan_count_latitud = df_conectividad_internet['Latitud'].isna().sum()
nan_count_longitud = df_conectividad_internet['Longitud'].isna().sum()

print("Cantidad de valores NaN en la columna 'Latitud':", nan_count_latitud)
print("Cantidad de valores NaN en la columna 'Longitud':", nan_count_longitud)

Cantidad de valores NaN en la columna 'Latitud': 0
Cantidad de valores NaN en la columna 'Longitud': 0


In [71]:
# Filtramos los valores nulos
df_filtered = df_conectividad_internet.dropna(subset=['Latitud', 'Longitud'])

In [80]:
# Agregar marcadores en el mapa para cada ubicación con conexión disponible
for _, fila in df_conexiones.iterrows():
    latitud = float(fila['Latitud']) if fila['Latitud'] != '' else 0.0
    longitud = float(fila['Longitud']) if fila['Longitud'] != '' else 0.0
    localidad = fila['Localidad']
    provincia = fila['Provincia']
    
    # Determinar el ícono según la conexión disponible
    if fila['ADSL'] == 'SI':
        icono = folium.Icon(color='red')
    elif fila['CABLEMODEM'] == 'SI':
        icono = folium.Icon(color='blue')
    elif fila['FIBRAOPTICA'] == 'SI':
        icono = folium.Icon(color='green')
    elif fila['SATELITAL'] == 'SI':
        icono = folium.Icon(color='purple')
    else:
        icono = folium.Icon(color='gray')

    # Crear un marcador con información emergente y el ícono correspondiente
    marcador = folium.Marker(location=[latitud, longitud],
                             popup=f'Localidad: {localidad}\nProvincia: {provincia}',
                             icon=icono)
    
    # Agregar el marcador al mapa
    marcador.add_to(mapa)

# Mostrar el mapa
mapa

In [ ]:
#def asignar_color(valor):
 #   if valor == "SI":
  #      return "green"
   # elif valor == "NO":
    #    return "red"
    #else:
      #  return "gray"

In [ ]:
# for index, row in df_conectividad_internet.iterrows():
  #  lat = row['Latitud']
   # lon = row['Longitud']
    #label = f"{row['Provincia']}, {row['Localidad']}"

    # Verificar si los valores de latitud y longitud no son NaN
    #if pd.notnull(lat) and pd.notnull(lon):
        # Crear un diccionario de estilos con colores asignados para cada columna
     #   estilo = {
      #      'ADSL': asignar_color(row['ADSL']),
       #     'CABLEMODEM': asignar_color(row['CABLEMODEM']),
        #    'DIALUP': asignar_color(row['DIALUP']),
         #   'FIBRAOPTICA': asignar_color(row['FIBRAOPTICA']),
          #  'SATELITAL': asignar_color(row['SATELITAL']),
           # 'WIRELESS': asignar_color(row['WIRELESS']),
           # 'TELEFONIAFIJA': asignar_color(row['TELEFONIAFIJA']),
            #'3G': asignar_color(row['3G']),
            #'4G': asignar_color(row['4G'])
        #}

        # Crear un ícono con los estilos de colores asignados
        #icono = folium.Icon(color='gray', icon='info-sign', icon_color='white', **estilo)

        # Agregar el marcador al mapa con el ícono correspondiente
        #folium.Marker(location=[lat, lon], popup=label, icon=icono).add_to(mapa)